In [1]:
!git clone https://github.com/am1tyadav/superhero

Cloning into 'superhero'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), 47.06 KiB | 5.23 MiB/s, done.


In [2]:
with open('superhero/superheroes.txt', 'r') as f:
  data = f.read()
data[:100]

'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [3]:
import tensorflow as tf

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters= '!"#$%&()*+-/,:;<=>?@[\\]^_{\}',
    split ='\n'
)

In [5]:
tokenizer.fit_on_texts(data)

In [6]:
char_to_index.keys()

NameError: ignored

In [7]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k, v in char_to_index.items())
print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


In [9]:
names[1]

NameError: ignored

In [10]:
names = data.splitlines()
names[:9]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t']

In [11]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [12]:
def name_to_seq(name) :
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [28]:
name_to_seq('a')

[2]

In [14]:
def seq_to_name(seq) :
  return ''.join([index_to_char[i] for i in seq if i!=0])

In [15]:
seq_to_name(name_to_seq(names[0]))

'jumpa\t'

creating and padding sequences

In [16]:
sequences = []
for name in names: 
  seq = name_to_seq(name)
  if len(seq)>=2:
    sequences+= [seq[:i] for i in range(2, len(seq)+1)]

In [17]:
sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [18]:
max_len = max([len(x) for x in sequences])
max_len

33

In [19]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding = 'pre',
    maxlen = max_len
)
padded_sequences[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 25, 16],
      dtype=int32)

In [20]:
padded_sequences.shape

(88279, 33)

In [21]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]

In [22]:
x.shape,y.shape

((88279, 32), (88279,))

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y)


In [24]:
num_chars = len(char_to_index.keys()) + 1

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM, Bidirectional, Dense

model = Sequential(
    [
        Embedding(num_chars, 8, input_length = max_len-1),
        Conv1D(64,5, strides=1, activation='tanh', padding = 'causal'), # filter =64, kernel_size = 5, strides = =1
        MaxPool1D(2),
        LSTM(32),
        Dense(num_chars, activation = 'softmax')
     ]
)

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 8)             232       
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            2624      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 29)                957       
                                                                 
Total params: 16,229
Trainable params: 16,229
Non-trainable params: 0
____________________________________________________

In [26]:
h = model.fit(
    x_train,y_train
    , validation_data = (x_test, y_test),
    epochs = 50, verbose = 2,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ]
)

Epoch 1/50
2070/2070 - 21s - loss: 2.7297 - accuracy: 0.1894 - val_loss: 2.5847 - val_accuracy: 0.2348 - 21s/epoch - 10ms/step
Epoch 2/50
2070/2070 - 11s - loss: 2.5265 - accuracy: 0.2391 - val_loss: 2.4978 - val_accuracy: 0.2463 - 11s/epoch - 5ms/step
Epoch 3/50
2070/2070 - 10s - loss: 2.4657 - accuracy: 0.2542 - val_loss: 2.4527 - val_accuracy: 0.2623 - 10s/epoch - 5ms/step
Epoch 4/50
2070/2070 - 10s - loss: 2.4190 - accuracy: 0.2709 - val_loss: 2.4173 - val_accuracy: 0.2717 - 10s/epoch - 5ms/step
Epoch 5/50
2070/2070 - 11s - loss: 2.3782 - accuracy: 0.2793 - val_loss: 2.3855 - val_accuracy: 0.2808 - 11s/epoch - 5ms/step
Epoch 6/50
2070/2070 - 10s - loss: 2.3461 - accuracy: 0.2879 - val_loss: 2.3605 - val_accuracy: 0.2847 - 10s/epoch - 5ms/step
Epoch 7/50
2070/2070 - 10s - loss: 2.3179 - accuracy: 0.2949 - val_loss: 2.3444 - val_accuracy: 0.2886 - 10s/epoch - 5ms/step
Epoch 8/50
2070/2070 - 11s - loss: 2.2936 - accuracy: 0.3036 - val_loss: 2.3267 - val_accuracy: 0.2967 - 11s/epoch - 

In [36]:
def generate_names(seed) :
  for i in range(0, 40):
    seq = name_to_seq(seed)
    padded = tf.keras.preprocessing.sequence.pad_sequences([seq], padding = 'pre', maxlen = max_len-1, truncating = 'pre')
    pred = model.predict(padded)[0]
    pred_char = index_to_char[tf.argmax(pred).numpy()]
    seed+=pred_char

    if pred_char == '\t' :
      break

  print(seed)

In [42]:
generate_names('x')

1/1 [==============================] - 0s 17ms/step
xena sander	
